## Setting

In [2]:
import pandas as pd
import sys
import os
import numpy as np
from clust.pipeline import data_pipeline 

sys.path.append("../")
sys.path.append("../..")
sys.path.append("../../..")
sys.path.append("../../../..")

In [3]:
from clust.ingestion import influx_client

clust_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"lRO_RjCuIpdgzpwx9bwhU_0MhyFflFA_fcPRVy6nGLEmJPnFAjB4zt-xbUh6UCaAA_36jCpZ3s9KCj9KkggcRA==",
        "org":"clust"
}

cedp_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"U9QaDrozQDsGdnxpZkkq5NJ1fwzDefEDtR7a_vAaylcxR6LsY1wCc36QVVqDah6ytctNYFM__CfzFuzQ8K4Niw==",
        "org":"cedp"
}

agridata_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"jUOMSF1jfAg7Kh-DscoswyBfMe6VAN1XdEabpYSTDbkeMw8SdNf1KFmQD0bomJC2rmLy0dwvwLrnySZxCfoMUQ==",
        "org":"cedp"
}

db_client = influx_client.InfluxClient(cedp_config)

## Get Data

In [4]:
bk_name = 'farm_protectedHorticulture_vegetables_cucumber'
ms_name = 'environment_knu_sghDG'

In [5]:
data = db_client.get_data(bk_name,ms_name)
data

,INT_CO2,INT_REL_HD,MID_INT_TMP,SLR_RAD
time,,,,
2021-10-19 00:00:00+00:00,NaN,60.500000,19.800000,0.000000
2021-10-19 00:03:00+00:00,NaN,60.300000,61.724860,0.000000
2021-10-19 00:04:00+00:00,949.695615,NaN,65.942175,0.000000
2021-10-19 00:05:00+00:00,549.000000,147.403441,NaN,0.000000
2021-10-19 00:05:45+00:00,1043.913556,162.209287,19.800000,NaN
...,...,...,...,...
2022-05-02 23:55:00+00:00,503.000000,162.928453,14.700000,0.000000
2022-05-02 23:56:00+00:00,NaN,79.100000,49.295213,0.000000
2022-05-02 23:57:00+00:00,281.965362,167.546283,NaN,0.000000


## Set Parameter

In [6]:
# 측정 기준 정하기
range_limits = {
    'max_num': {
        'INT_CO2': 870, 'INT_REL_HD': 105, 'MID_INT_TMP': 40,'SLR_RAD': 900
    },
    'min_num': {
        'INT_CO2': 290, 'INT_REL_HD': 15, 'MID_INT_TMP': 9,'SLR_RAD': 0
    }
}

expected_types = {'INT_CO2': int, 'INT_REL_HD': float, 'MID_INT_TMP': float, 'SLR_RAD' : float}

error_values = {
    "INT_CO2": [9999, -9999],  
    "MID_INT_TMP": [9999, -9999],  
}

## Test

In [7]:
pipeline_freq = [
      ['data_refinement', 
        {'remove_duplication': {'flag': None}, 
        'static_frequency': {'flag': True, 'frequency': '60S'}}]
     ]
freq_data = data_pipeline.pipeline(data, pipeline_freq, expected_types=expected_types)

data_refinement
{'remove_duplication': {'flag': None}, 'static_frequency': {'flag': True, 'frequency': '60S'}}


/home/jhlee/CLUST_KETI/keti-cedp-clust/clust/preprocessing/refinement/frequency.py:120: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  data_static_frequency = data_static_frequency.resample(freq).mean()
/home/jhlee/CLUST_KETI/keti-cedp-clust/clust/preprocessing/refinement/frequency.py:123: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  data_static_frequency = data_static_frequency.asfreq(freq=freq)


(282240, 4)


In [11]:
freq_data = freq_data.replace({pd.NA: np.nan})
freq_data[freq_data.columns] = freq_data[freq_data.columns].astype("float64")
freq_data

,INT_CO2,INT_REL_HD,MID_INT_TMP,SLR_RAD
time,,,,
2021-10-19 00:00:00+00:00,NaN,60.500000,19.800000,0.000000
2021-10-19 00:01:00+00:00,NaN,NaN,NaN,NaN
2021-10-19 00:02:00+00:00,NaN,NaN,NaN,NaN
2021-10-19 00:03:00+00:00,NaN,60.300000,61.724860,0.000000
2021-10-19 00:04:00+00:00,950.0,NaN,65.942175,0.000000
...,...,...,...,...
2022-05-02 23:55:00+00:00,503.0,162.928453,14.700000,0.000000
2022-05-02 23:56:00+00:00,NaN,79.100000,49.295213,0.000000
2022-05-02 23:57:00+00:00,282.0,167.546283,NaN,0.000000
